# Hotel Rating Prediction
In this project we will inspect the question   
whether we can or can not predict the overall rating on a given hotel  
according to different parameters which we already know

## Data Acquisition 

At this step we are going to acquire the data we need from the website  
and create the DataFrame which will serve us throughout the project.

In [16]:
# First lets import the libraries we need :
import requests
import pandas as pd
import numpy as np
import scipy as sc
from bs4 import BeautifulSoup
import time
import os
import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# The next command will be responsible to display graphs in our notebook.
%matplotlib inline

### Crawling 

We will now fetch what we need from the website "Booking.com"

In [52]:
# creating the function to load a soup object 
def load_soup_object(link):
    response_link = requests.get(link ,headers=0)
    soup_object = BeautifulSoup(response_link.content, "html.parser")
    return soup_object

# Lets get the URL and parser it before we create our DataFrame
URL = "https://www.booking.com/he"
soup = load_soup_object(URL)

# Now we will create the lists that will serve as the columns for the DataFrame
Hotel_Name = []
Staff_Rating = []
Facilities_Rating = []
Cleanliness_Rating = []
Comfort_Rating = []
Value_For_Money_Rating = []
Location_Rating = []
Free_WiFi_Rating = []



# accessing the class that contains the list of regions - not only in israel 
# and creating a list of known cities to be extracted 
# counter will be used to save the index of the desired regions - then to specify the index withing the 'searches' list
searches = soup.find(class_="ia_section-container clearfix wcu-seo-regions-poi").find_all('a')
cities = ["אזור ירושלים","אזור המרכז","גליל","גליל עליון","כנרת","אזור תל אביב","דרום הארץ","ים המלח","נגב","צפון הארץ","רמת הגולן"]
names = []
indexes = []
href_list = []
counter = 0
# now, 'searches' is a list with all the regions,we want to loop through and extrace only the ones we want
# we are first getting the name of each object in the list, and checking if it exists in our desired regeions list 'cities'
# if term is met, we are appending its index(integer) to the indexes list
# and extracting the corresponding link with the "href" tag and concatanating it to the original URL
for i in searches:
    name1 = i.get_text().replace("\n", "") 
    if name1 in cities:
        names.append(name1)
        indexes.append(counter)
        href = searches[counter].get("href").strip()
        URL2 = URL+href
        soup2 = load_soup_object(URL2)
        
        # now lets parse it so we can inspect and work with each desired page 
        # each region`s page has a specific "Watch all" link that needs to be accessed
        # therefore, we are accessing the class that contains the only object(link) we need
        # and concatanating the link to the original URL
        
        searches2 = soup2.find(class_="lp-bui-section bui-spacer--largest x2").find_all("a")
        print("counter is : ", counter)
        print("region is : ", name1)
        nexthref = URL + searches2[0].get("href").strip()
        print("full name : ", searches2[0].get_text())
        href_list.append(nexthref)
        print("\nhref is :\n",nexthref, "\n")
    counter+=1

# now after this for loop we have a list that contains all the links to the full page of each desired region

print(names)
print(indexes)
print(names[0])
#print(href_list)


    
    

counter is :  1
region is :  צפון הארץ
full name :  

צפו בכל 1,509 המלונות ב-צפון הארץ



href is :
 https://www.booking.com/he/searchresults.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4AqatqpMGwAIB0gIkZDQyZjg2YzUtMmI5Ny00Yzk2LWFjY2MtZTgxOTRhNWUwZTE12AIG4AIB&sid=435bbc4d70945a0d42d7b9d57b81437e&region=3638& 

counter is :  3
region is :  דרום הארץ
full name :  

צפו בכל 901 המלונות ב-דרום הארץ



href is :
 https://www.booking.com/he/searchresults.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4AqatqpMGwAIB0gIkZDQyZjg2YzUtMmI5Ny00Yzk2LWFjY2MtZTgxOTRhNWUwZTE12AIG4AIB&sid=aaffb80871ec34abeec0db0763f5d6a3&region=3642& 

counter is :  4
region is :  אזור ירושלים
full name :  

צפו בכל 579 המלונות ב-אזור ירושלים



href is :
 https://www.booking.com/he/searchresults.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4AqatqpMGwAIB0gIkZDQyZjg2YzUtMmI5Ny00Yzk2LWFjY2MtZTgxOTRhNWUwZTE12AIG4AIB;sid

Now after we have a list of the correct and desired regions,  
we are going to access each region's page and loop through all other pages  
so we can extract the information we need regarding each item(hotel) in the page.

In [53]:
# now that we have all the links, we need to parse each one with html.parser so we can work with it
name_index = 0
for link in href_list:
    city_soup = load_soup_object(link)
    number_of_pages = int(city_soup.find(class_="d493c719bc").find_all('li')[-1].get_text())
    print("number of pages in ", names[name_index], " is: " , number_of_pages)
    name_index += 1
    for i in range(number_of_pages):
        
        
        pass
        
    

number of pages in  צפון הארץ  is:  40
number of pages in  דרום הארץ  is:  40
number of pages in  אזור ירושלים  is:  20
number of pages in  נגב  is:  39
number of pages in  גליל  is:  40
number of pages in  גליל עליון  is:  31
number of pages in  אזור המרכז  is:  40
number of pages in  ים המלח  is:  6
number of pages in  רמת הגולן  is:  15
number of pages in  כנרת  is:  15
number of pages in  אזור תל אביב  is:  40
